In [ ]:
import sys
from pathlib import Path
import torch

ROOT_PATH=Path("..").resolve().absolute()
DEVICE=torch.device("cuda")#torch.device("cpu")
sys.path.append(str(ROOT_PATH))

In [ ]:
import main.module_base
from audio_diffusion_pytorch import LogNormalDistribution
import torch

diffusion_sigma_distribution = LogNormalDistribution(mean=-3.0, std=1.0)
    
model_1 = main.module_base.Model(
    learning_rate=1e-4,
    beta1=0.9,
    beta2=0.99,
    in_channels=2,
    channels=256,
    patch_factor=32,
    patch_blocks=1,
    resnet_groups=8,
    kernel_multiplier_downsample=2,
    kernel_sizes_init=[1, 3, 7],
    multipliers=[1, 2, 4, 4, 4, 4, 4],
    factors=[4, 4, 4, 2, 2, 2],
    num_blocks= [2, 2, 2, 2, 2, 2],
    attentions= [False, False, False, True, True, True],
    attention_heads=8,
    attention_features=128,
    attention_multiplier=2,
    use_nearest_upsample=False,
    use_skip_scale=True,
    use_attention_bottleneck=True,
    diffusion_sigma_distribution=diffusion_sigma_distribution,
    diffusion_sigma_data=0.2,
    diffusion_dynamic_threshold=0.0,
)


#ckpt = torch.load(ROOT_PATH / "data/checkpoints/piano.ckpt", map_location=DEVICE)
ckpt = torch.load("../logs/ckpts/2022-11-25-15-45-59/epoch=1285-valid_loss=0.018.ckpt", map_location=DEVICE)

#ckpt = torch.load("../logs/ckpts/2022-11-17-15-56-09/epoch=208-valid_loss=0.053.ckpt", map_location=DEVICE)
model_1.load_state_dict(ckpt["state_dict"])

In [ ]:
import main.module_base
from audio_diffusion_pytorch import LogNormalDistribution
import torch

diffusion_sigma_distribution = LogNormalDistribution(mean=-3.0, std=1.0)
    
model_2 = main.module_base.Model(
    learning_rate=1e-4,
    beta1=0.9,
    beta2=0.99,
    in_channels=1,
    channels=256,
    patch_factor=16,
    patch_blocks=1,
    resnet_groups=8,
    kernel_multiplier_downsample=2,
    kernel_sizes_init=[1, 3, 7],
    multipliers=[1, 2, 4, 4, 4, 4, 4],
    factors=[4, 4, 4, 2, 2, 2],
    num_blocks= [2, 2, 2, 2, 2, 2],
    attentions= [False, False, False, True, True, True],
    attention_heads=8,
    attention_features=128,
    attention_multiplier=2,
    use_nearest_upsample=False,
    use_skip_scale=True,
    use_attention_bottleneck=True,
    diffusion_sigma_distribution=diffusion_sigma_distribution,
    diffusion_sigma_data=0.2,
    diffusion_dynamic_threshold=0.0,
)

ckpt = torch.load(ROOT_PATH / "data/checkpoints/piano.ckpt", map_location=DEVICE)
#ckpt = torch.load("/data/MusDB/checkpoints/drums/drums-musdb.ckpt", map_location=DEVICE)
#ckpt = torch.load("../logs/ckpts/2022-11-22-18-14-22/epoch=10571-valid_loss=0.200.ckpt", map_location=DEVICE)
model_2.load_state_dict(ckpt["state_dict"])

In [ ]:
import torchaudio
from IPython.display import Audio
import math
from audio_data_pytorch.transforms import Loudness
sampling_rate = 22050

# @markdown Generation length in seconds (will be rounded to be a power of 2 of sample_rate*length)
#length = 10
length_samples = 262144#2**math.ceil(math.log2(length * sampling_rate))

# @markdown Number of samples to generate
num_samples = 1

# @markdown Number of diffusion steps (higher tends to be better but takes longer to generate)
num_steps = 10

#s1, sr1 = torchaudio.load('/data/MusDB/data/test_MusDB_reduced/Al James - Schoolboy Facination/bass.wav')
#s2, sr2 = torchaudio.load('/data/MusDB/data/test_MusDB_reduced/Al James - Schoolboy Facination/drums.wav')

  #s1, sr1 = torchaudio.load('/data/5B/Bass_22050/test/Track01881.wav')
#s2, sr2 = torchaudio.load('/data/5B/Piano_22050/test/Track01881.wav')
s1, sr1 = torchaudio.load('/data/MusDB/data/test_MusDB_reduced/Angels In Amplifiers - I\'m Alright/bass.wav')
s2, sr2 = torchaudio.load('/data/MusDB/data/test_MusDB_reduced/Angels In Amplifiers - I\'m Alright/drums.wav')

if not (sr1 == sr2 == sampling_rate):
    print(f"Inconsistent sampling rate: {sr1}, {sr2}, {sampling_rate}")

s1 = torchaudio.functional.resample(s1, orig_freq=sr1, new_freq=sampling_rate)
s2 = torchaudio.functional.resample(s2, orig_freq=sr2, new_freq=sampling_rate)
#s1 = Loudness(sampling_rate,-20.0)(s1)
#s2 = Loudness(sampling_rate, -20.0)(s2)

#convert to mono
s1, s2 = s1.mean(dim=0, keepdims=True), s2.mean(dim=0, keepdims=True)

# display(Audio(s1, rate = sampling_rate))
# display(Audio(s2, rate = sampling_rate))

start_sample = 100 * sampling_rate
s1 = s1.reshape(1, 1, -1)[:, :, start_sample:start_sample + length_samples]
s2 = s2.reshape(1, 1, -1)[:, :, start_sample:start_sample + length_samples]
m = s1+s2
#m = m[:, :, start_sample:start_sample + length_samples]

def plot_waves(s1, s2):
    import matplotlib.pyplot as plt
    #plt.plot(s1.reshape(-1))
    #plt.plot(s2.reshape(-1))
    
    fig = plt.figure()
    axes = fig.add_subplot(2,1,1)
    spec, _, _, img = plt.specgram(s1.reshape(-1), NFFT=1024,)

    axes = fig.add_subplot(2,1,2)
    spec, _, _, img = plt.specgram(s2.reshape(-1), NFFT=1024,)

    #plt.legend(["piano", "bass"])
    plt.show()
    
    display(Audio(s1.reshape(1,-1), rate = sampling_rate))
    display(Audio(s2.reshape(1,-1), rate = sampling_rate))
    display(Audio((s1+s2).reshape(1,-1), rate = sampling_rate))
    
plot_waves(s1, s2)

spec = torchaudio.transforms.Spectrogram(n_fft=1024, power=None)
invspec = torchaudio.transforms.InverseSpectrogram(n_fft=1024)
ts = torchaudio.transforms.TimeStretch(1024/2, n_freq=513, fixed_rate=1.4)
#plot_waves(invspec(ts(spec(s1))),s1)

In [ ]:
from typing import List, Optional

import torch
from torch import Tensor

from audio_diffusion_pytorch.diffusion import AEulerSampler, ADPM2Sampler, Diffusion, KarrasSchedule, Sampler, Schedule
from audio_diffusion_pytorch.model import AudioDiffusionModel
from audio_diffusion_pytorch.utils import default, exists

class DiffusionSeparator:
    def __init__(
        self,
        diffusions: List[Diffusion],
        *,
        samplers: List[Sampler],
        sigma_schedules: List[Schedule],
        num_steps: Optional[int] = None,
        nu = 0.2#2e-5 / 0.01**2
    ):
        super().__init__()
        self.denoise_fns = [denoise_fn for denoise_fn in diffusions]
        self.samplers = samplers
        self.sigma_schedules = sigma_schedules
        self.num_steps = num_steps
        self.nu = nu

    def forward(self, m: Tensor, noises: List[Tensor], num_steps: Optional[int] = None) -> List[Tensor]:
        device = noises[0].device
        num_steps = default(num_steps, self.num_steps)
        assert exists(num_steps), "Parameter `num_steps` must be provided"
        
        # Compute sigmas using schedule
        sigmas_list = [sigma_schedule(num_steps, device) for sigma_schedule in self.sigma_schedules]
        
        # Append additional kwargs to denoise function (used e.g. for conditional unet)
        fns = self.denoise_fns
        
        # Separation procedure
        xs = [sigmas[0] * noise for sigmas, noise in zip(sigmas_list, noises)]
                
        #likelihood_steps = torch.tensor([self.nu]*len(sigmas_list[0]))
        #likelihood_steps = compute_likelihood_steps(sigmas_list[0], gamma=0.0)

        @torch.no_grad()
        def perform_sample_step(xs:list, step:int):
            for j, x in enumerate(xs):
                    yield self.samplers[j].step(
                        x, fn=fns[j], sigma=sigmas_list[j][step], sigma_next=sigmas_list[j][step + 1])
                    
        # Denoise to sample
        for step in range(num_steps - 1):
            us = list(perform_sample_step(xs, step))
            g_x = torch.stack(xs).sum(dim=0)
            
            step_size = self.nu
            likelihood = step_size*(m - g_x)
            
            if step % 100 == 0:
                likelihood_norm = torch.norm(m - g_x)
                print("likelihood norm:", likelihood_norm.item())
                print("step-size:", step_size)
                print("prior 1 norm:", torch.norm(us[0] - xs[0]).item())
                print("prior 2 norm:", torch.norm(us[1] - xs[1]).item())
                print("")
                plot_waves(xs[0].cpu(), xs[1].cpu())
            
            for i in range(len(xs)):
                xs[i] = us[i] + likelihood
                
        xs = [x.clamp(-1.0, 1.0) for x in xs]
        print(likelihood_norm.item())
        return xs


@torch.no_grad()
def separate(
    model1,
    model2,
    mixture, 
    device: torch.device = torch.device("cuda"), 
    num_steps:int = 402,
):
    
    batch, in_channels = 1, 1
    samples = mixture.shape[-1]

    m = torch.tensor(mixture).to(device)
    models = [model1.model, model2.model]
    
    for model in models:
        model.to(device)
        
    #schedule = lambda num_steps,device: torch.flip(torch.sort(diffusion_sigma_distribution(num_steps*10, device))[0][::10],dims=[0])
    #schedule = lambda num_steps, device: torch.arange(1.0, 0.0, -1.0/num_steps,device=device)
    schedule = KarrasSchedule(sigma_min=1e-4, sigma_max=1.0, rho=9.0)
    
    fn = lambda x, sigma: model_1.model.diffusion.denoise_fn(x.repeat(1,2,1), sigma=sigma).mean()
    
    diffusion_separator = DiffusionSeparator(
        [fn, model_2.model.diffusion.denoise_fn],#[model.diffusion.denoise_fn for model in models],
        samplers=[ADPM2Sampler(rho=1.0), ADPM2Sampler(rho=1.0)],
        sigma_schedules=[schedule, schedule],
        num_steps=num_steps
    )
    noises = [torch.randn_like(m).to(device), torch.randn_like(m).to(device)]
    return diffusion_separator.forward(m, noises)


y1, y2 = separate(model_1, model_2, m)

In [ ]:
display(Audio(y1.detach().cpu().view(1,-1), rate = sampling_rate))
display(Audio(y2.detach().cpu().view(1,-1), rate = sampling_rate))
display(Audio((y1+y2).detach().cpu().view(1,-1), rate = sampling_rate))
display(Audio((m).detach().cpu().view(1,-1), rate = sampling_rate))

import torchmetrics.functional.audio as tma
def si_snr(preds, targets):
    return tma.scale_invariant_signal_noise_ratio(preds.cpu(), targets.cpu()).item()

y1 = y1.mean(dim=1, keepdims=True)
y2 = y2.mean(dim=1, keepdims=True)
print(f"SI-SNR (1): {si_snr(y1, s1)}")
print(f"SI-SNR (2): {si_snr(y2, s2)}")
print(f"SI-SNRi(1): {si_snr(y1, s1) - si_snr(s1, m)}")
print(f"SI-SNRi (2): {si_snr(y2, s2) - si_snr(s2, m)}")
print(f"SI-SNR (mix): {si_snr(y1+y2, m)}")

plot_waves(y1.cpu(), y2.cpu())
plot_waves((s1.cpu()-y1.cpu()), (s2.cpu()-y2.cpu()))

In [ ]:
torchaudio.save("s1.wav", s1.reshape(1,-1).cpu(), 22050)
torchaudio.save("s2.wav", s2.reshape(1,-1).cpu(), 22050)
torchaudio.save("m.wav", m.reshape(1,-1).cpu(), 22050)

# Data Separation

In [ ]:
from audio_data_pytorch import WAVDataset, AllTransform

dataset_1 = WAVDataset(
    "/data/5B/Piano_22050/test",
    transforms=AllTransform(source_rate=22050, target_rate=22050,random_crop_size=262144, loudness=-20),
)

dataset_2 = WAVDataset(
    "/data/5B/Bass_22050/test",
    transforms=AllTransform(source_rate=22050, target_rate=22050, random_crop_size=262144, loudness=-20),
)

In [ ]:
from pathlib import Path
import tqdm

output_folder = Path("separations")
output_folder.mkdir(exist_ok=True)
for i in tqdm.autonotebook.tqdm(range(min(len(dataset_1), len(dataset_2)))):
    x1, x2 = dataset_1[i], dataset_2[i]
    x1 ,x2 = x1.unsqueeze(0), x2.unsqueeze(0)
    
    assert len(x1.shape) == len(x2.shape) == 3
    assert x1.shape[0:2] == x2.shape[0:2] == (1,1)
    assert x1.shape == x2.shape
    
    y1, y2 = separate(model_1, model_2, x1 + x2)
    
    separation_folder = (output_folder / f"{i}" )
    separation_folder.mkdir(exist_ok=True)
    
    torchaudio.save(separation_folder/ "ori1.wav", x1.reshape(1,-1).cpu(), 22050)
    torchaudio.save(separation_folder/"ori2.wav", x2.reshape(1,-1).cpu(), 22050)
    torchaudio.save(separation_folder /"sep1.wav", y1.reshape(1,-1).cpu(), 22050)
    torchaudio.save(separation_folder /"sep2.wav", y2.reshape(1,-1).cpu(), 22050)
    torchaudio.save(separation_folder /"mix.wav", m.reshape(1,-1).cpu(), 22050)


# Other stuff

In [ ]:
from audio_diffusion_pytorch.diffusion import AEulerSampler, ADPM2Sampler, Diffusion, KarrasSchedule, Sampler, Schedule

with torch.no_grad():
     samples = model_1.model.sample(
         noise=torch.randn((num_samples, 2, length_samples), device=DEVICE),
         num_steps=100,
         sigma_schedule=KarrasSchedule(sigma_min=1e-4, sigma_max=3.0, rho=7.0),
         sampler=ADPM2Sampler(),
     )

# Log audio samples
for i, sample in enumerate(samples):
     display(Audio(sample.cpu(), rate = sampling_rate))

In [ ]:
k = KarrasSchedule(sigma_min=1e-4, sigma_max=3.0, rho=9.0)
sigmas_rev = k(num_steps=99, device="cpu")

#step_size = torch.tensor([3e-1]*100)

import matplotlib.pyplot as plt
#plt.plot(sigmas)
plt.plot(compute_likelihood_steps(sigmas_rev, gamma=0.0128))
#plt.plot(step_size)

In [ ]:
from audio_diffusion_pytorch.diffusion import KarrasSchedule
import matplotlib.pyplot as plt
import numpy as np

def plot_wave(s1):    
    fig = plt.figure()
    axes = fig.add_subplot(2,1,1)
    spec, _, _, img = plt.specgram(s1.reshape(-1), NFFT=1024,)
    plt.show()
    
    display(Audio(s1.reshape(1,-1), rate = sampling_rate))

k = KarrasSchedule(sigma_min=1e-4, sigma_max=3.0, rho=9.0)
sigmas = k(num_steps=10, device="cpu")

plt.plot(sigmas)
noise = torch.randn_like(s1).cpu()

for sigma in sigmas:
    print(f"({sigma}) SI-SNR : {si_snr(sigma*noise + s1, s1)}")
    #plot_wave(sigma*noise + s1)
    print("-----------------------\n")

In [ ]:
x=np.arange(100)/1000
y=10*np.log10(np.linalg.norm(s1)**2 / (s1.numel()*x**2))
#plt.plot(x,y)
import plotly.express as px
px.line(y)

In [ ]:
import math
from IPython.display import Audio
from audio_diffusion_pytorch.diffusion import ADPM2Sampler, KarrasSchedule
sampling_rate=22050

device="cuda"
with torch.no_grad():
    
    model = model_2.model
    model.to(device)
        
    samples = model.sample(
         noise=torch.randn((1, 1, 2**math.ceil(math.log2(10 * sampling_rate))), device='cuda'),
         num_steps=100,
         sigma_schedule=KarrasSchedule(
             sigma_min=1e-4,
             sigma_max=1.0,
             rho=9.0
         ),
         sampler=ADPM2Sampler(rho=1.0),
     )

# Log audio samples
for i, sample in enumerate(samples):
    display(Audio(sample.cpu(), rate = sampling_rate))